# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

## Question 1 rough work

In [428]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

In [429]:
nhl_df=pd.read_csv("nhl.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


In [430]:
def removenote(cities, tag):
    templist = []
    for a in cities[tag]:
        templist.append(a.split('[')[0])
    cities[tag] = templist
    return cities

In [431]:
cities = removenote(cities,'NHL')
# cities = removenote(cities,'NFL')
# cities = removenote(cities,'MLB')
# cities = removenote(cities,'NBA')

cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [432]:
cities = cities.rename(columns={'Population (2016 est.)[8]':'Population 2016'})
cities.head()

,Metropolitan area,Population 2016,NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [433]:
cities = cities.drop(['NFL','MLB','NBA'],axis=1)

In [434]:
cities[cities['Metropolitan area'] == 'New York City']['NHL'][0]

'RangersIslandersDevils'

In [435]:
s = 'RangersIslandersDevils'

s = 'Red Wings'
pattern = '([A-Z][^\s][a-z]*)'
x = re.findall(pattern,cities[cities['Metropolitan area'] == 'New York City']['NHL'][0])
x

['Rangers', 'Islanders', 'Devils']

In [436]:
# replace new york city teams
pattern = '([A-Z][^\s][a-z]*)'
x = re.findall(pattern,cities[cities['Metropolitan area'] == 'New York City']['NHL'][0])
cities.iloc[0]['NHL'] = x
cities.head()

,Metropolitan area,Population 2016,NHL
0,New York City,20153634,"[Rangers, Islanders, Devils]"
1,Los Angeles,13310447,KingsDucks
2,San Francisco Bay Area,6657982,Sharks
3,Chicago,9512999,Blackhawks
4,Dallas–Fort Worth,7233323,Stars


In [437]:
# replace LA teams
pattern = '([A-Z][^\s][a-z]*)'
x = re.findall(pattern,cities[cities['Metropolitan area'] == 'Los Angeles']['NHL'][1])
cities.iloc[1]['NHL'] = x
cities.head()

,Metropolitan area,Population 2016,NHL
0,New York City,20153634,"[Rangers, Islanders, Devils]"
1,Los Angeles,13310447,"[Kings, Ducks]"
2,San Francisco Bay Area,6657982,Sharks
3,Chicago,9512999,Blackhawks
4,Dallas–Fort Worth,7233323,Stars


In [438]:
cities = cities.explode('NHL')

In [439]:
cities.head()

,Metropolitan area,Population 2016,NHL
0,New York City,20153634,Rangers
0,New York City,20153634,Islanders
0,New York City,20153634,Devils
1,Los Angeles,13310447,Kings
1,Los Angeles,13310447,Ducks


In [440]:
# cities['NHL'].to_list()

In [441]:
# remove the unnecessary first row
nhl_df = nhl_df.drop(0)
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
1,Tampa Bay Lightning*,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins*,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs*,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL


In [442]:
nhl_df['team'] = nhl_df['team'].replace('*','')

In [443]:
nhl_df['team'] = nhl_df['team'].apply(lambda x: x.replace('*',''))
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
1,Tampa Bay Lightning,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Boston Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Toronto Maple Leafs,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
5,Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL


In [444]:
# nhl_df[nhl_df['year'] == 2018]

In [445]:
all_nhl_teams = set(nhl_df['team'].to_list())
all_nhl_teams

{'Anaheim Ducks',
 'Arizona Coyotes',
 'Atlantic Division',
 'Boston Bruins',
 'Buffalo Sabres',
 'Calgary Flames',
 'Carolina Hurricanes',
 'Central Division',
 'Chicago Blackhawks',
 'Colorado Avalanche',
 'Columbus Blue Jackets',
 'Dallas Stars',
 'Detroit Red Wings',
 'Edmonton Oilers',
 'Florida Panthers',
 'Los Angeles Kings',
 'Metropolitan Division',
 'Minnesota Wild',
 'Montreal Canadiens',
 'Nashville Predators',
 'New Jersey Devils',
 'New York Islanders',
 'New York Rangers',
 'Ottawa Senators',
 'Pacific Division',
 'Philadelphia Flyers',
 'Phoenix Coyotes',
 'Pittsburgh Penguins',
 'San Jose Sharks',
 'St. Louis Blues',
 'Tampa Bay Lightning',
 'Toronto Maple Leafs',
 'Vancouver Canucks',
 'Vegas Golden Knights',
 'Washington Capitals',
 'Winnipeg Jets'}

In [446]:
# Now fix the team names in cities df
def findorgname(team):
    if team == '':
        return '-'
    for x in all_nhl_teams:
        if team in x:
            return x
    return '-'
temp_nhl_list = []
for team in cities['NHL']:
    temp_nhl_list.append(findorgname(team))

# change the nhl columns in cities df
cities['NHL'] = temp_nhl_list

In [447]:
nhl_df = nhl_df.set_index('team')
nhl_df.head()

,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
team,,,,,,,,,,,,,,
Tampa Bay Lightning,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
Boston Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
Toronto Maple Leafs,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
Florida Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
Detroit Red Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL


In [448]:
cities = cities.rename({'NHL':'team'}, axis = 1)
cities = cities.set_index('team')
cities.head()

,Metropolitan area,Population 2016
team,,
New York Rangers,New York City,20153634
New York Islanders,New York City,20153634
New Jersey Devils,New York City,20153634
Los Angeles Kings,Los Angeles,13310447
Anaheim Ducks,Los Angeles,13310447


In [449]:
merged = pd.merge(nhl_df, cities, how='inner',left_index = True, right_index=True)
merged.head()

,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,Metropolitan area,Population 2016
team,,,,,,,,,,,,,,,,
Anaheim Ducks,82,44,25,13,101,.616,235,216,0.24,0.01,.555,40,2018,NHL,Los Angeles,13310447
Anaheim Ducks,82,46,23,13,105,.640,223,200,0.24,-0.04,.561,43,2017,NHL,Los Angeles,13310447
Anaheim Ducks,82,46,25,11,103,.628,218,192,0.27,-0.05,.567,43,2017,NHL,Los Angeles,13310447
Anaheim Ducks,82,51,24,7,109,.665,236,226,0.10,-0.03,.604,43,2015,NHL,Los Angeles,13310447
Anaheim Ducks,82,54,20,8,116,.707,266,209,0.68,-0.01,.677,51,2014,NHL,Los Angeles,13310447


In [450]:
# removing all other than year 2018
merged = merged[merged['year'] == 2018]
merged.head()

,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,Metropolitan area,Population 2016
team,,,,,,,,,,,,,,,,
Anaheim Ducks,82,44,25,13,101,.616,235,216,0.24,0.01,.555,40,2018,NHL,Los Angeles,13310447
Boston Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL,Boston,4794447
Buffalo Sabres,82,25,45,12,62,.378,199,280,-0.98,0.01,.311,24,2018,NHL,Buffalo,1132804
Calgary Flames,82,37,35,10,84,.512,218,248,-0.33,0.03,.470,35,2018,NHL,Calgary,1392609
Carolina Hurricanes,82,36,35,11,83,.506,228,256,-0.35,-0.01,.439,33,2018,NHL,Raleigh,1302946


In [451]:
area_indexed = merged.reset_index()
area_indexed.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League,Metropolitan area,Population 2016
0,Anaheim Ducks,82,44,25,13,101,.616,235,216,0.24,0.01,.555,40,2018,NHL,Los Angeles,13310447
1,Boston Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL,Boston,4794447
2,Buffalo Sabres,82,25,45,12,62,.378,199,280,-0.98,0.01,.311,24,2018,NHL,Buffalo,1132804
3,Calgary Flames,82,37,35,10,84,.512,218,248,-0.33,0.03,.470,35,2018,NHL,Calgary,1392609
4,Carolina Hurricanes,82,36,35,11,83,.506,228,256,-0.35,-0.01,.439,33,2018,NHL,Raleigh,1302946


In [452]:
area_indexed = area_indexed.drop(area_indexed.columns.difference(['W','L','team','Population 2016','Metropolitan area']),axis=1)

In [453]:
area_indexed = area_indexed.astype({'W':int, 'L':int,'Population 2016':int})

In [454]:
# I was trying to group them without removing other years
# resulting = []
# for group, frame in area_indexed.groupby('Metropolitan area'):
#     print(group)
#     population = frame['Population 2016'][0]
#     win = np.sum(frame['W'])
#     print('\tWon matches are',win)    
#     loss = np.sum(frame['L'])
#     print('\tLost matches are',loss)
#     print('\twin/loss ratio is ',win/(loss+win))
#     print('\tPopulation is ',population)

## Previous implementation - Doesn't work. Answer is mentioned below

In [455]:
# group by city, sum up the win loss

a = 0
win_loss_by_region = []
population_by_region = []
for group, frame in area_indexed.groupby('Metropolitan area'):
#     print('\n',group,'----------')
#     print('\n',frame)
    win = np.sum(frame['W'])
    loss = np.sum(frame['L'])
    win_loss_by_region.append(win/(loss+win))

    population = frame['Population 2016'].iloc[0]
    population_by_region.append(population)
#     print('len {} and type {}'.format(len(population),type(population)))
#     if len(population) >1:
#         population = frame['Population 2016'][0]
#     print(group, win, loss, population)
    a+=1
# print(win_loss_by_region)
# print(population_by_region)
print('len',len(population_by_region))
print(stats.pearsonr(population_by_region, win_loss_by_region))

len 27
(0.01337397844453335, 0.9472124883027294)


# Answer to question 1

In [456]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("nhl.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]


def removenote(cities, tag):
    templist = []
    for a in cities[tag]:
        templist.append(a.split('[')[0])
    cities[tag] = templist
    return cities

# Now fix the team names in cities df
def findorgname(team,all_nhl_teams):
    if team == '':
        return '-'
    for x in all_nhl_teams:
        if team in x:
            return x
    return '-'

def nhl_correlation():
    global cities, nhl_df
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    # cities = removenote(cities,'NHL')
    templist = []
    # print(cities['NHL'])
    my_cities = cities.copy()
    my_nhl_df = nhl_df.copy()
    for a in my_cities['NHL']:
        templist.append(a.split('[')[0])
    my_cities['NHL'] = templist
    
    my_cities = my_cities.rename(columns={'Population (2016 est.)[8]':'Population 2016'})
    my_cities = my_cities.drop(['NFL','MLB','NBA'],axis=1)
    
    # replace new york city teams
    pattern = '([A-Z][^\s][a-z]*)'
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'New York City']['NHL'][0])
    my_cities.iloc[0]['NHL'] = x
    
    # replace LA teams
    pattern = '([A-Z][^\s][a-z]*)'
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'Los Angeles']['NHL'][1])
    my_cities.iloc[1]['NHL'] = x
    
    
#     my_nhl_df = my_nhl_df.drop(0)
    
    my_nhl_df['team'] = my_nhl_df['team'].apply(lambda x: x.replace('*',''))
    
    all_nhl_teams = set(my_nhl_df['team'].to_list())
    my_cities = my_cities.explode('NHL')
    
    temp_nhl_list = []
    for team in my_cities['NHL']:
        actual_name = findorgname(team,all_nhl_teams)
        if actual_name == 'Phoenix Coyotes':
            actual_name = 'Arizona Coyotes'
        temp_nhl_list.append(actual_name)

    # change the nhl columns in my_cities df
    my_cities['NHL'] = temp_nhl_list
    my_nhl_df = my_nhl_df.set_index('team')
    
    my_cities = my_cities.rename({'NHL':'team'}, axis = 1)
    my_cities = my_cities.set_index('team')
    
    merged = pd.merge(my_cities,my_nhl_df, how='inner',left_index = True, right_index=True)
    merged = merged[merged['year'] == 2018]
    area_indexed = merged.reset_index()
    area_indexed = area_indexed.drop(area_indexed.columns.difference(['W','L','team','Population 2016','Metropolitan area']),axis=1)
    area_indexed = area_indexed.astype({'W':int, 'L':int,'Population 2016':int})
    area_indexed['win_loss_ratio'] = area_indexed['W']/(area_indexed['W']+area_indexed['L'])
    
    # group by city, sum up the win loss
    a = 0
    for group, frame in area_indexed.groupby('Metropolitan area'):
        # win = np.sum(frame['W'])
        # loss = np.sum(frame['L'])
        win_loss_by_region.append(np.mean(frame['win_loss_ratio']))

        population = frame['Population 2016'].iloc[0]
        population_by_region.append(population)
        a+=1

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [457]:
nhl_correlation()

0.012486162921209923

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

## Question 2 rough work

In [458]:
nba_df=pd.read_csv("nba.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [459]:
nba_df[:]

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League
0,Toronto Raptors* (1),59,23,0.72,—,111.7,103.9,7.29,2018,NBA
1,Boston Celtics* (2),55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA
2,Philadelphia 76ers* (3),52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA
3,Cleveland Cavaliers* (4),50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA
4,Indiana Pacers* (5),48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA
...,...,...,...,...,...,...,...,...,...,...
157,San Antonio Spurs* (1),62,20,.756,—,105.4,97.6,8.00,2014,NBA
158,Houston Rockets* (4),54,28,.659,8.0,107.7,103.1,5.06,2014,NBA
159,Memphis Grizzlies* (7),50,32,.610,12.0,96.1,94.6,2.18,2014,NBA
160,Dallas Mavericks* (8),49,33,.598,13.0,104.8,102.4,2.91,2014,NBA


In [460]:
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [461]:
my_cities = cities.copy()
my_nba_df = nba_df.copy()

In [462]:
my_nba_df['team'] = my_nba_df['team'].apply(lambda x: x.split('*')[0]).apply(lambda x: x.split('\xa0(')[0]).to_list()

# remove note 
templist = []
for a in my_cities['NBA']:
    templist.append(a.split('[')[0])
my_cities['NBA'] = templist
my_cities = my_cities.rename(columns={'Population (2016 est.)[8]':'Population 2016'})
my_cities = my_cities.drop(['NFL','MLB','NHL'],axis=1)

In [463]:
# replace new york city teams
pattern = '([A-Z][^\s][a-z]*)'
x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'New York City']['NBA'][0])
my_cities.iloc[0]['NBA'] = x

# replace Los Angeles teams
x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'Los Angeles']['NBA'][1])
my_cities.iloc[1]['NBA'] = x

# explode into multiple rows
my_cities = my_cities.explode('NBA')

In [464]:
all_nba_teams = set(my_nba_df['team'].to_list())
temp_nba_list = []
for team in my_cities['NBA']:
    actual_name = findorgname(team,all_nba_teams)
    temp_nba_list.append(actual_name)
# change the nba columns in my_cities df
my_cities['NBA'] = temp_nba_list


my_nba_df = my_nba_df.set_index('team')
    
my_cities = my_cities.rename({'NBA':'team'}, axis = 1)
my_cities = my_cities.set_index('team')

In [465]:
merged = pd.merge(my_cities,my_nba_df, how='inner',left_index = True, right_index=True)
merged = merged[merged['year'] == 2018]
area_indexed = merged.reset_index()

In [466]:
area_indexed = area_indexed.drop(area_indexed.columns.difference(['W','L','team','Population 2016','Metropolitan area']),axis=1)

In [467]:
area_indexed = area_indexed.astype({'W':int, 'L':int,'Population 2016':int})
area_indexed['win_loss_ratio'] = area_indexed['W']/(area_indexed['W']+area_indexed['L'])
a = 1

population_by_region = [] # pass in metropolitan area population from cities
win_loss_by_region = []

for group, frame in area_indexed.groupby('Metropolitan area'):
    # win = np.sum(frame['W'])
    # loss = np.sum(frame['L'])
    win_loss_by_region.append(np.mean(frame['win_loss_ratio']))

    population = frame['Population 2016'].iloc[0]
    population_by_region.append(population)
    a+=1

# Answer to Q2

In [468]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("nba.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    my_cities = cities.copy()
    my_nba_df = nba_df.copy()
    
    my_nba_df['team'] = my_nba_df['team'].apply(lambda x: x.split('*')[0]).apply(lambda x: x.split('\xa0(')[0]).to_list()

    # remove note 
    templist = []
    for a in my_cities['NBA']:
        templist.append(a.split('[')[0])
    my_cities['NBA'] = templist
    my_cities = my_cities.rename(columns={'Population (2016 est.)[8]':'Population 2016'})
    my_cities = my_cities.drop(['NFL','MLB','NHL'],axis=1)
    
    # replace new york city teams
    pattern = '([A-Z][^\s][a-z]*)'
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'New York City']['NBA'][0])
    my_cities.iloc[0]['NBA'] = x

    # replace Los Angeles teams
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'Los Angeles']['NBA'][1])
    my_cities.iloc[1]['NBA'] = x

    # explode into multiple rows
    my_cities = my_cities.explode('NBA')
    
    all_nba_teams = set(my_nba_df['team'].to_list())
    temp_nba_list = []
    for team in my_cities['NBA']:
        actual_name = findorgname(team,all_nba_teams)
        temp_nba_list.append(actual_name)
    # change the nba columns in my_cities df
    my_cities['NBA'] = temp_nba_list


    my_nba_df = my_nba_df.set_index('team')

    my_cities = my_cities.rename({'NBA':'team'}, axis = 1)
    my_cities = my_cities.set_index('team')
    
    merged = pd.merge(my_cities,my_nba_df, how='inner',left_index = True, right_index=True)
    merged = merged[merged['year'] == 2018]
    area_indexed = merged.reset_index()
    
    area_indexed = area_indexed.drop(area_indexed.columns.difference(['W','L','team','Population 2016','Metropolitan area']),axis=1)
    
    area_indexed = area_indexed.astype({'W':int, 'L':int,'Population 2016':int})
    area_indexed['win_loss_ratio'] = area_indexed['W']/(area_indexed['W']+area_indexed['L'])
    a = 1

    for group, frame in area_indexed.groupby('Metropolitan area'):
        # win = np.sum(frame['W'])
        # loss = np.sum(frame['L'])
        win_loss_by_region.append(np.mean(frame['win_loss_ratio']))

        population = frame['Population 2016'].iloc[0]
        population_by_region.append(population)
        a+=1
    

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

-0.17657160252844614

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

# Question 3 Rough work

In [469]:
mlb_df=pd.read_csv("mlb.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [470]:
my_cities = cities.copy()
my_mlb_df = mlb_df.copy()

In [471]:
my_mlb_df.head()

,team,W,L,W-L%,GB,year,League
0,Boston Red Sox,108,54,0.667,--,2018,MLB
1,New York Yankees,100,62,0.617,8.0,2018,MLB
2,Tampa Bay Rays,90,72,0.556,18.0,2018,MLB
3,Toronto Blue Jays,73,89,0.451,35.0,2018,MLB
4,Baltimore Orioles,47,115,0.290,61.0,2018,MLB


In [472]:
my_cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [473]:
# remove note 
templist = []
for a in my_cities['MLB']:
    templist.append(a.split('[')[0])
my_cities['MLB'] = templist

my_cities = my_cities.rename(columns={'Population (2016 est.)[8]':'Population 2016'})
my_cities = my_cities.drop(['NFL','NBA','NHL'],axis=1)

# replace new york city teams
pattern = '([A-Z][^\s][a-z]*)'

x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'New York City']['MLB'][0])
my_cities.iloc[0]['MLB'] = x

# replace Los Angeles teams
x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'Los Angeles']['MLB'][1])
my_cities.iloc[1]['MLB'] = x

# replace San Francisco teams
x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'San Francisco Bay Area']['MLB'][2])
my_cities.iloc[2]['MLB'] = x

# replace Chicago teams
x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'Chicago']['MLB'][3])
y = []
y.append(x[0])
y.append(x[1]+' '+x[2])
y
my_cities.iloc[3]['MLB'] = y

# explode into multiple rows
my_cities = my_cities.explode('MLB')

In [474]:
all_mlb_teams = set(my_mlb_df['team'].to_list())
temp_mlb_list = []
for team in my_cities['MLB']:
    actual_name = findorgname(team,all_mlb_teams)
    temp_mlb_list.append(actual_name)
# change the nba columns in my_cities df
my_cities['MLB'] = temp_mlb_list

my_mlb_df = my_mlb_df.set_index('team')

my_cities = my_cities.rename({'MLB':'team'}, axis = 1)
my_cities = my_cities.set_index('team')

merged = pd.merge(my_cities,my_mlb_df, how='inner',left_index = True, right_index=True)
merged = merged[merged['year'] == 2018]
area_indexed = merged.reset_index()

area_indexed = area_indexed.drop(area_indexed.columns.difference(['W','L','team','Population 2016','Metropolitan area']),axis=1)

area_indexed = area_indexed.astype({'W':int, 'L':int,'Population 2016':int})
area_indexed['win_loss_ratio'] = area_indexed['W']/(area_indexed['W']+area_indexed['L'])
a = 1


population_by_region = [] # pass in metropolitan area population from cities
win_loss_by_region = []
    
for group, frame in area_indexed.groupby('Metropolitan area'):
    # win = np.sum(frame['W'])
    # loss = np.sum(frame['L'])
    win_loss_by_region.append(np.mean(frame['win_loss_ratio']))

    population = frame['Population 2016'].iloc[0]
    population_by_region.append(population)
    a+=1

In [475]:
len(win_loss_by_region)

26

# Solution to Question 3

In [795]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("mlb.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    raise NotImplementedError()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)

In [796]:

def mlb_correlation(): 
    global mlb_df, cities
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
    
    my_cities = cities.copy()
    my_mlb_df = mlb_df.copy()
    
    # remove note 
    templist = []
    for a in my_cities['MLB']:
        templist.append(a.split('[')[0])
    my_cities['MLB'] = templist

    my_cities = my_cities.rename(columns={'Population (2016 est.)[8]':'Population 2016'})
    my_cities = my_cities.drop(['NFL','NBA','NHL'],axis=1)

    # replace new york city teams
    pattern = '([A-Z][^\s][a-z]*)'

    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'New York City']['MLB'][0])
    my_cities.iloc[0]['MLB'] = x

    # replace Los Angeles teams
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'Los Angeles']['MLB'][1])
    my_cities.iloc[1]['MLB'] = x

    # replace San Francisco teams
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'San Francisco Bay Area']['MLB'][2])
    my_cities.iloc[2]['MLB'] = x

    # replace Chicago teams
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'Chicago']['MLB'][3])
    y = []
    y.append(x[0])
    y.append(x[1]+' '+x[2])
    my_cities.iloc[3]['MLB'] = y

    # explode into multiple rows
    my_cities = my_cities.explode('MLB')
    
    all_mlb_teams = set(my_mlb_df['team'].to_list())
    temp_mlb_list = []
    for team in my_cities['MLB']:
        actual_name = findorgname(team,all_mlb_teams)
        temp_mlb_list.append(actual_name)
    # change the nba columns in my_cities df
    my_cities['MLB'] = temp_mlb_list

    my_mlb_df = my_mlb_df.set_index('team')

    my_cities = my_cities.rename({'MLB':'team'}, axis = 1)
    my_cities = my_cities.set_index('team')

    merged = pd.merge(my_cities,my_mlb_df, how='inner',left_index = True, right_index=True)
    merged = merged[merged['year'] == 2018]
    area_indexed = merged.reset_index()

    area_indexed = area_indexed.drop(area_indexed.columns.difference(['W','L','team','Population 2016','Metropolitan area']),axis=1)

    area_indexed = area_indexed.astype({'W':int, 'L':int,'Population 2016':int})
    area_indexed['win_loss_ratio'] = area_indexed['W']/(area_indexed['W']+area_indexed['L'])
    a = 1


    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = []

    for group, frame in area_indexed.groupby('Metropolitan area'):
        # win = np.sum(frame['W'])
        # loss = np.sum(frame['L'])
        win_loss_by_region.append(np.mean(frame['win_loss_ratio']))

        population = frame['Population 2016'].iloc[0]
        population_by_region.append(population)
        a+=1

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

# Question 4 Rought work

In [797]:
nfl_df=pd.read_csv("nfl.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

In [798]:
def nfl_correlation():
    global nfl_df, cities
    
    my_cities = cities.copy()
    my_nfl_df = nfl_df.copy()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    
    my_cities = cities.copy()
    my_nfl_df = nfl_df.copy()
    
    my_nfl_df['team'] = my_nfl_df['team'].apply(lambda x: x.split('*')[0]).apply(lambda x: x.split('+')[0]).to_list()
    
    # remove note 
    templist = []
    for a in my_cities['NFL']:
        templist.append(a.split('[')[0])
    my_cities['NFL'] = templist

    my_cities = my_cities.rename(columns={'Population (2016 est.)[8]':'Population 2016'})
    my_cities = my_cities.drop(['MLB','NBA','NHL'],axis=1)

    # replace new york city teams
    pattern = '([A-Z][^\s][a-z]*)'

    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'New York City']['NFL'][0])
    my_cities.iloc[0]['NFL'] = x

    # replace Los Angeles teams
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'Los Angeles']['NFL'][1])
    my_cities.iloc[1]['NFL'] = x

    # replace San Francisco teams
    patternx = '([0-9A-Z][^\s][a-z]*)'
    x = re.findall(patternx,my_cities[my_cities['Metropolitan area'] == 'San Francisco Bay Area']['NFL'][2])
    my_cities.iloc[2]['NFL'] = x

    # explode into multiple rows
    my_cities = my_cities.explode('NFL')
    
    all_nfl_teams = set(my_nfl_df['team'].to_list())
    temp_nfl_list = []
    for team in my_cities['NFL']:
        actual_name = findorgname(team,all_nfl_teams)
        temp_nfl_list.append(actual_name)
    # change the nba columns in my_cities df
    my_cities['NFL'] = temp_nfl_list

    my_nfl_df = my_nfl_df.set_index('team')

    my_cities = my_cities.rename({'NFL':'team'}, axis = 1)
    my_cities = my_cities.set_index('team')

    merged = pd.merge(my_cities,my_nfl_df, how='inner',left_index = True, right_index=True)
    merged = merged[merged['year'] == 2018]
    area_indexed = merged.reset_index()

    area_indexed = area_indexed.drop(area_indexed.columns.difference(['W','L','team','Population 2016','Metropolitan area']),axis=1)

    area_indexed = area_indexed.astype({'W':int, 'L':int,'Population 2016':int})
    area_indexed['win_loss_ratio'] = area_indexed['W']/(area_indexed['W']+area_indexed['L'])
    a = 1
    
    for group, frame in area_indexed.groupby('Metropolitan area'):
        # win = np.sum(frame['W'])
        # loss = np.sum(frame['L'])
        win_loss_by_region.append(np.mean(frame['win_loss_ratio']))

        population = frame['Population 2016'].iloc[0]
        population_by_region.append(population)
        a+=1
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

-0.008854721884910915

In [587]:
my_cities = cities.copy()
my_nfl_df = nfl_df.copy()

In [588]:
my_cities.head()
# remove note

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets[note 1],YankeesMets[note 2],KnicksNets,RangersIslandersDevils[note 3]
1,Los Angeles,13310447,RamsChargers[note 4],DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders[note 6],GiantsAthletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],CubsWhite Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars


In [589]:
population_by_region = [] # pass in metropolitan area population from cities
win_loss_by_region = []

In [591]:
my_cities = cities.copy()
my_nfl_df = nfl_df.copy()
my_nfl_df['team'] = my_nfl_df['team'].apply(lambda x: x.split('*')[0]).apply(lambda x: x.split('+')[0]).to_list()

# remove note 
templist = []
for a in my_cities['NFL']:
    templist.append(a.split('[')[0])
my_cities['NFL'] = templist

my_cities = my_cities.rename(columns={'Population (2016 est.)[8]':'Population 2016'})
my_cities = my_cities.drop(['MLB','NBA','NHL'],axis=1)

# replace new york city teams
pattern = '([A-Z][^\s][a-z]*)'

x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'New York City']['NFL'][0])
my_cities.iloc[0]['NFL'] = x

# replace Los Angeles teams
x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'Los Angeles']['NFL'][1])
my_cities.iloc[1]['NFL'] = x

# replace San Francisco teams
patternx = '([0-9A-Z][^\s][a-z]*)'
x = re.findall(patternx,my_cities[my_cities['Metropolitan area'] == 'San Francisco Bay Area']['NFL'][2])
my_cities.iloc[2]['NFL'] = x

# explode into multiple rows
my_cities = my_cities.explode('NFL')

all_nfl_teams = set(my_nfl_df['team'].to_list())
temp_nfl_list = []
for team in my_cities['NFL']:
    actual_name = findorgname(team,all_nfl_teams)
    temp_nfl_list.append(actual_name)
# change the nba columns in my_cities df
my_cities['NFL'] = temp_nfl_list

my_nfl_df = my_nfl_df.set_index('team')

my_cities = my_cities.rename({'NFL':'team'}, axis = 1)
my_cities = my_cities.set_index('team')

merged = pd.merge(my_cities,my_nfl_df, how='inner',left_index = True, right_index=True)

merged = merged[merged['year'] == 2018]
area_indexed = merged.reset_index()

area_indexed = area_indexed.drop(area_indexed.columns.difference(['W','L','team','Population 2016','Metropolitan area']),axis=1)

area_indexed = area_indexed.astype({'W':int, 'L':int,'Population 2016':int})
area_indexed['win_loss_ratio'] = area_indexed['W']/(area_indexed['W']+area_indexed['L'])
a = 1



    
for group, frame in area_indexed.groupby('Metropolitan area'):
    print(a,group,np.mean(frame['win_loss_ratio']))
    # win = np.sum(frame['W'])
    # loss = np.sum(frame['L'])
    win_loss_by_region.append(np.mean(frame['win_loss_ratio']))

    population = frame['Population 2016'].iloc[0]
    population_by_region.append(population)
    a+=1
    

1 Atlanta 0.4375
2 Baltimore 0.625
3 Boston 0.6875
4 Buffalo 0.375
5 Charlotte 0.4375
6 Chicago 0.75
7 Cincinnati 0.375
8 Cleveland 0.4666666666666667
9 Dallas–Fort Worth 0.625
10 Denver 0.375
11 Detroit 0.375
12 Green Bay 0.4
13 Houston 0.6875
14 Indianapolis 0.625
15 Jacksonville 0.3125
16 Kansas City 0.75
17 Los Angeles 0.75
18 Miami–Fort Lauderdale 0.4375
19 Minneapolis–Saint Paul 0.5333333333333333
20 Nashville 0.5625
21 New Orleans 0.8125
22 New York City 0.28125
23 Philadelphia 0.5625
24 Phoenix 0.1875
25 Pittsburgh 0.6
26 San Francisco Bay Area 0.25
27 Seattle 0.625
28 Tampa Bay Area 0.3125
29 Washington, D.C. 0.4375


In [593]:
stats.pearsonr(population_by_region, win_loss_by_region)[0]

-0.008854721884910915

# Solution to Question 4

In [790]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("nfl.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# def nfl_correlationx(): 
#     # YOUR CODE HERE
#     raise NotImplementedError()
    
#     population_by_region = [] # pass in metropolitan area population from cities
#     win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

#     assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
#     assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

#     return stats.pearsonr(population_by_region, win_loss_by_region)

In [791]:
def nfl_correlation():
    
    my_cities = cities.copy()
    my_nfl_df = nfl_df.copy()
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    
    my_cities = cities.copy()
    my_nfl_df = nfl_df.copy()
    
    my_nfl_df['team'] = my_nfl_df['team'].apply(lambda x: x.split('*')[0]).apply(lambda x: x.split('+')[0]).to_list()
    
    # remove note 
    templist = []
    for a in my_cities['NFL']:
        templist.append(a.split('[')[0])
    my_cities['NFL'] = templist

    my_cities = my_cities.rename(columns={'Population (2016 est.)[8]':'Population 2016'})
    my_cities = my_cities.drop(['MLB','NBA','NHL'],axis=1)

    # replace new york city teams
    pattern = '([A-Z][^\s][a-z]*)'

    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'New York City']['NFL'][0])
    my_cities.iloc[0]['NFL'] = x

    # replace Los Angeles teams
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'Los Angeles']['NFL'][1])
    my_cities.iloc[1]['NFL'] = x

    # replace San Francisco teams
    patternx = '([0-9A-Z][^\s][a-z]*)'
    x = re.findall(patternx,my_cities[my_cities['Metropolitan area'] == 'San Francisco Bay Area']['NFL'][2])
    my_cities.iloc[2]['NFL'] = x

    # explode into multiple rows
    my_cities = my_cities.explode('NFL')
    
    all_nfl_teams = set(my_nfl_df['team'].to_list())
    temp_nfl_list = []
    for team in my_cities['NFL']:
        actual_name = findorgname(team,all_nfl_teams)
        temp_nfl_list.append(actual_name)
    # change the nba columns in my_cities df
    my_cities['NFL'] = temp_nfl_list

    my_nfl_df = my_nfl_df.set_index('team')

    my_cities = my_cities.rename({'NFL':'team'}, axis = 1)
    my_cities = my_cities.set_index('team')

    merged = pd.merge(my_cities,my_nfl_df, how='inner',left_index = True, right_index=True)
    merged = merged[merged['year'] == 2018]
    area_indexed = merged.reset_index()

    area_indexed = area_indexed.drop(area_indexed.columns.difference(['W','L','team','Population 2016','Metropolitan area']),axis=1)

    area_indexed = area_indexed.astype({'W':int, 'L':int,'Population 2016':int})
    area_indexed['win_loss_ratio'] = area_indexed['W']/(area_indexed['W']+area_indexed['L'])
    a = 1
    
    for group, frame in area_indexed.groupby('Metropolitan area'):
        # win = np.sum(frame['W'])
        # loss = np.sum(frame['L'])
        win_loss_by_region.append(np.mean(frame['win_loss_ratio']))

        population = frame['Population 2016'].iloc[0]
        population_by_region.append(population)
        a+=1
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    
    
    
    

In [792]:
nfl_correlation()

-0.008854721884910915

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [756]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("mlb.csv")
nhl_df=pd.read_csv("nhl.csv")
nba_df=pd.read_csv("nba.csv")
nfl_df=pd.read_csv("nfl.csv")
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

# def sports_team_performance():
    
#     # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
#     # df.loc["NFL","NFL"] should return np.nan
#     sports = ['NFL', 'NBA', 'NHL', 'MLB']
#     p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
#     assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NFL p-value should be around 0.02"
#     assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
#     return p_values

In [757]:
sports = ['NFL', 'NBA', 'NHL', 'MLB']
p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

In [758]:
def getnhldf(nhl_df, cities):
    templist = []
    # print(cities['NHL'])
    my_cities = cities.copy()
    my_nhl_df = nhl_df.copy()
    for a in my_cities['NHL']:
        templist.append(a.split('[')[0])
    my_cities['NHL'] = templist
    
    my_cities = my_cities.rename(columns={'Population (2016 est.)[8]':'Population 2016'})
    my_cities = my_cities.drop(['NFL','MLB','NBA'],axis=1)
    
    # replace new york city teams
    pattern = '([A-Z][^\s][a-z]*)'
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'New York City']['NHL'][0])
    my_cities.iloc[0]['NHL'] = x
    
    # replace LA teams
    pattern = '([A-Z][^\s][a-z]*)'
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'Los Angeles']['NHL'][1])
    my_cities.iloc[1]['NHL'] = x
    
    
#     my_nhl_df = my_nhl_df.drop(0)
    
    my_nhl_df['team'] = my_nhl_df['team'].apply(lambda x: x.replace('*',''))
    
    all_nhl_teams = set(my_nhl_df['team'].to_list())
    my_cities = my_cities.explode('NHL')
    
    temp_nhl_list = []
    for team in my_cities['NHL']:
        actual_name = findorgname(team,all_nhl_teams)
        if actual_name == 'Phoenix Coyotes':
            actual_name = 'Arizona Coyotes'
        temp_nhl_list.append(actual_name)

    # change the nhl columns in my_cities df
    my_cities['NHL'] = temp_nhl_list
    my_nhl_df = my_nhl_df.set_index('team')
    
    my_cities = my_cities.rename({'NHL':'team'}, axis = 1)
    my_cities = my_cities.set_index('team')
    
    merged = pd.merge(my_cities,my_nhl_df, how='inner',left_index = True, right_index=True)
    merged = merged[merged['year'] == 2018]
    area_indexed = merged.reset_index()
    
    area_indexed = area_indexed.drop(area_indexed.columns.difference(['W','L','team','Population 2016','Metropolitan area']),axis=1)
    area_indexed = area_indexed.astype({'W':int, 'L':int,'Population 2016':int})
    area_indexed['win_loss_ratio'] = area_indexed['W']/(area_indexed['W']+area_indexed['L'])
    
    result_list = []
    for group, frame in area_indexed.groupby('Metropolitan area'):
        pop = frame['Population 2016'].iloc[0]
        win_loss = np.mean(frame['win_loss_ratio'])
        item = {
                'Metropolitan area':group,
                'Population 2016':pop,
                'win_loss_nhl':win_loss
               }
        result_list.append(item)
        
    
    return pd.DataFrame(result_list)

In [759]:
def getnbadf(nba_df, cities):
    my_cities = cities.copy()
    my_nba_df = nba_df.copy()
    
    my_nba_df['team'] = my_nba_df['team'].apply(lambda x: x.split('*')[0]).apply(lambda x: x.split('\xa0(')[0]).to_list()

    # remove note 
    templist = []
    for a in my_cities['NBA']:
        templist.append(a.split('[')[0])
    my_cities['NBA'] = templist
    my_cities = my_cities.rename(columns={'Population (2016 est.)[8]':'Population 2016'})
    my_cities = my_cities.drop(['NFL','MLB','NHL'],axis=1)
    
    # replace new york city teams
    pattern = '([A-Z][^\s][a-z]*)'
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'New York City']['NBA'][0])
    my_cities.iloc[0]['NBA'] = x

    # replace Los Angeles teams
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'Los Angeles']['NBA'][1])
    my_cities.iloc[1]['NBA'] = x

    # explode into multiple rows
    my_cities = my_cities.explode('NBA')
    
    all_nba_teams = set(my_nba_df['team'].to_list())
    temp_nba_list = []
    for team in my_cities['NBA']:
        actual_name = findorgname(team,all_nba_teams)
        temp_nba_list.append(actual_name)
    # change the nba columns in my_cities df
    my_cities['NBA'] = temp_nba_list


    my_nba_df = my_nba_df.set_index('team')

    my_cities = my_cities.rename({'NBA':'team'}, axis = 1)
    my_cities = my_cities.set_index('team')
    
    merged = pd.merge(my_cities,my_nba_df, how='inner',left_index = True, right_index=True)
    merged = merged[merged['year'] == 2018]
    area_indexed = merged.reset_index()
    
    area_indexed = area_indexed.drop(area_indexed.columns.difference(['W','L','team','Population 2016','Metropolitan area']),axis=1)
    
    area_indexed = area_indexed.astype({'W':int, 'L':int,'Population 2016':int})
    area_indexed['win_loss_ratio'] = area_indexed['W']/(area_indexed['W']+area_indexed['L'])
    a = 1

    result_list = []
    for group, frame in area_indexed.groupby('Metropolitan area'):
        pop = frame['Population 2016'].iloc[0]
        win_loss = np.mean(frame['win_loss_ratio'])
        item = {
                'Metropolitan area':group,
                'Population 2016':pop,
                'win_loss_nba':win_loss
               }
        result_list.append(item)
        
    
    return pd.DataFrame(result_list)

In [760]:
def getmlbdf(mlb_df, cities):
    my_cities = cities.copy()
    my_mlb_df = mlb_df.copy()
    
    # remove note 
    templist = []
    for a in my_cities['MLB']:
        templist.append(a.split('[')[0])
    my_cities['MLB'] = templist

    my_cities = my_cities.rename(columns={'Population (2016 est.)[8]':'Population 2016'})
    my_cities = my_cities.drop(['NFL','NBA','NHL'],axis=1)

    # replace new york city teams
    pattern = '([A-Z][^\s][a-z]*)'

    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'New York City']['MLB'][0])
    my_cities.iloc[0]['MLB'] = x

    # replace Los Angeles teams
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'Los Angeles']['MLB'][1])
    my_cities.iloc[1]['MLB'] = x

    # replace San Francisco teams
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'San Francisco Bay Area']['MLB'][2])
    my_cities.iloc[2]['MLB'] = x

    # replace Chicago teams
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'Chicago']['MLB'][3])
    y = []
    y.append(x[0])
    y.append(x[1]+' '+x[2])
    my_cities.iloc[3]['MLB'] = y

    # explode into multiple rows
    my_cities = my_cities.explode('MLB')
    
    all_mlb_teams = set(my_mlb_df['team'].to_list())
    temp_mlb_list = []
    for team in my_cities['MLB']:
        actual_name = findorgname(team,all_mlb_teams)
        temp_mlb_list.append(actual_name)
    # change the nba columns in my_cities df
    my_cities['MLB'] = temp_mlb_list

    my_mlb_df = my_mlb_df.set_index('team')

    my_cities = my_cities.rename({'MLB':'team'}, axis = 1)
    my_cities = my_cities.set_index('team')

    merged = pd.merge(my_cities,my_mlb_df, how='inner',left_index = True, right_index=True)
    merged = merged[merged['year'] == 2018]
    area_indexed = merged.reset_index()

    area_indexed = area_indexed.drop(area_indexed.columns.difference(['W','L','team','Population 2016','Metropolitan area']),axis=1)

    area_indexed = area_indexed.astype({'W':int, 'L':int,'Population 2016':int})
    area_indexed['win_loss_ratio'] = area_indexed['W']/(area_indexed['W']+area_indexed['L'])
    a = 1
    
    result_list = []
    for group, frame in area_indexed.groupby('Metropolitan area'):
        pop = frame['Population 2016'].iloc[0]
        win_loss = np.mean(frame['win_loss_ratio'])
        item = {
                'Metropolitan area':group,
                'Population 2016':pop,
                'win_loss_mlb':win_loss
               }
        result_list.append(item)
        
    
    return pd.DataFrame(result_list)

In [761]:
def getnfldf(nfl_df, cities):
    my_cities = cities.copy()
    my_nfl_df = nfl_df.copy()
    
    my_nfl_df['team'] = my_nfl_df['team'].apply(lambda x: x.split('*')[0]).apply(lambda x: x.split('+')[0]).to_list()
    
    # remove note 
    templist = []
    for a in my_cities['NFL']:
        templist.append(a.split('[')[0])
    my_cities['NFL'] = templist

    my_cities = my_cities.rename(columns={'Population (2016 est.)[8]':'Population 2016'})
    my_cities = my_cities.drop(['MLB','NBA','NHL'],axis=1)

    # replace new york city teams
    pattern = '([A-Z][^\s][a-z]*)'

    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'New York City']['NFL'][0])
    my_cities.iloc[0]['NFL'] = x

    # replace Los Angeles teams
    x = re.findall(pattern,my_cities[my_cities['Metropolitan area'] == 'Los Angeles']['NFL'][1])
    my_cities.iloc[1]['NFL'] = x

    # replace San Francisco teams
    patternx = '([0-9A-Z][^\s][a-z]*)'
    x = re.findall(patternx,my_cities[my_cities['Metropolitan area'] == 'San Francisco Bay Area']['NFL'][2])
    my_cities.iloc[2]['NFL'] = x

    # explode into multiple rows
    my_cities = my_cities.explode('NFL')
    
    all_nfl_teams = set(my_nfl_df['team'].to_list())
    temp_nfl_list = []
    for team in my_cities['NFL']:
        actual_name = findorgname(team,all_nfl_teams)
        temp_nfl_list.append(actual_name)
    # change the nba columns in my_cities df
    my_cities['NFL'] = temp_nfl_list

    my_nfl_df = my_nfl_df.set_index('team')

    my_cities = my_cities.rename({'NFL':'team'}, axis = 1)
    my_cities = my_cities.set_index('team')

    merged = pd.merge(my_cities,my_nfl_df, how='inner',left_index = True, right_index=True)
    merged = merged[merged['year'] == 2018]
    area_indexed = merged.reset_index()

    area_indexed = area_indexed.drop(area_indexed.columns.difference(['W','L','team','Population 2016','Metropolitan area']),axis=1)

    area_indexed = area_indexed.astype({'W':int, 'L':int,'Population 2016':int})
    area_indexed['win_loss_ratio'] = area_indexed['W']/(area_indexed['W']+area_indexed['L'])
    a = 1
    
    result_list = []
    for group, frame in area_indexed.groupby('Metropolitan area'):
        pop = frame['Population 2016'].iloc[0]
        win_loss = np.mean(frame['win_loss_ratio'])
        item = {
                'Metropolitan area':group,
                'Population 2016':pop,
                'win_loss_nfl':win_loss
               }
        result_list.append(item)
        
    
    return pd.DataFrame(result_list)

In [762]:

nhl_data = getnhldf(nhl_df, cities)

nba_data = getnbadf(nba_df, cities)

mlb_data = getmlbdf(mlb_df, cities)

nfl_data = getnfldf(nfl_df, cities)

In [763]:
nhl_data = nhl_data.set_index('Metropolitan area')
nba_data = nba_data.set_index('Metropolitan area')
mlb_data = mlb_data.set_index('Metropolitan area')
nfl_data = nfl_data.set_index('Metropolitan area')
print()

In [764]:
nhl_data.head()

,Population 2016,win_loss_nhl
Metropolitan area,,
Boston,4794447,0.714286
Buffalo,1132804,0.357143
Calgary,1392609,0.513889
Chicago,9512999,0.458333
Columbus,2041520,0.600000


In [765]:
nba_data.head()

,Population 2016,win_loss_nba
Metropolitan area,,
Atlanta,5789700,0.292683
Boston,4794447,0.670732
Charlotte,2474314,0.439024
Chicago,9512999,0.329268
Cleveland,2055612,0.609756


In [766]:
mlb_data.head()

,Population 2016,win_loss_mlb
Metropolitan area,,
Atlanta,5789700,0.555556
Baltimore,2798886,0.290123
Boston,4794447,0.666667
Chicago,9512999,0.482769
Cincinnati,2165139,0.413580


In [767]:
nfl_data.head()

,Population 2016,win_loss_nfl
Metropolitan area,,
Atlanta,5789700,0.4375
Baltimore,2798886,0.6250
Boston,4794447,0.6875
Buffalo,1132804,0.3750
Charlotte,2474314,0.4375


In [768]:
from scipy.stats import ttest_ind

In [769]:
nhl_nfl = pd.merge(nhl_data,nfl_data, how='inner',left_index = True, right_index=True)
nhl_nfl_pval = ttest_ind(nhl_nfl['win_loss_nhl'],nhl_nfl['win_loss_nfl'])
nhl_nfl_pval

Ttest_indResult(statistic=2.0826111677009105, pvalue=0.04536336635359821)

In [771]:
# nhl_nfl.head()

In [772]:
nhl_mlb = pd.merge(nhl_data,mlb_data, how='inner',left_index = True, right_index=True)
nhl_mlb_pval = ttest_ind(nhl_mlb['win_loss_nhl'],nhl_mlb['win_loss_mlb'])
nhl_mlb_pval

Ttest_indResult(statistic=3.18502578259423, pvalue=0.003220274466194636)

In [773]:
nhl_nba = pd.merge(nhl_data,nba_data, how='inner',left_index = True, right_index=True)
nhl_nba_pval = ttest_ind(nhl_nba['win_loss_nhl'],nhl_nba['win_loss_nba'])
nhl_nba_pval

Ttest_indResult(statistic=1.5050389212319741, pvalue=0.14436675651787648)

In [774]:
nfl_nba = pd.merge(nfl_data,nba_data, how='inner',left_index = True, right_index=True)
nfl_nba_pval = ttest_ind(nfl_nba['win_loss_nfl'],nfl_nba['win_loss_nba'])
nfl_nba_pval

Ttest_indResult(statistic=0.034912368928248616, pvalue=0.9723424843947726)

In [775]:
nfl_mlb = pd.merge(nfl_data,mlb_data, how='inner',left_index = True, right_index=True)
nfl_mlb_pval = ttest_ind(nfl_mlb['win_loss_nfl'],nfl_mlb['win_loss_mlb'])
nfl_mlb_pval

Ttest_indResult(statistic=0.22726841904361372, pvalue=0.821317695710284)

In [776]:
nba_mlb = pd.merge(nba_data,mlb_data, how='inner',left_index = True, right_index=True)
nba_mlb_pval = ttest_ind(nba_mlb['win_loss_nba'],nba_mlb['win_loss_mlb'])
nba_mlb_pval

Ttest_indResult(statistic=0.05452923959368028, pvalue=0.9568326769522106)

In [777]:
ttest_ind(nba_data['win_loss_nba'],nba_data['win_loss_nba'])

Ttest_indResult(statistic=0.0, pvalue=1.0)

In [778]:
p_values

,NFL,NBA,NHL,MLB
NFL,NaN,NaN,NaN,NaN
NBA,NaN,NaN,NaN,NaN
NHL,NaN,NaN,NaN,NaN
MLB,NaN,NaN,NaN,NaN


In [779]:
p_values.loc["NBA", "NHL"] = nhl_nba_pval[1]
p_values.loc["NHL", "NBA"] = nhl_nba_pval[1]

p_values.loc["MLB", "NHL"] = nhl_mlb_pval[1]
p_values.loc["NHL", "MLB"] = nhl_mlb_pval[1]

p_values.loc["NFL", "NHL"] = nhl_nfl_pval[1]
p_values.loc["NHL", "NFL"] = nhl_nfl_pval[1]

p_values.loc["NFL", "NBA"] = nfl_nba_pval[1]
p_values.loc["NBA", "NFL"] = nfl_nba_pval[1]

p_values.loc["MLB", "NFL"] = nfl_mlb_pval[1]
p_values.loc["NFL", "MLB"] = nfl_mlb_pval[1]

p_values.loc["NBA", "MLB"] = nba_mlb_pval[1]
p_values.loc["MLB", "NBA"] = nba_mlb_pval[1]

In [780]:
p_values = p_values.fillna(1.0)
p_values

,NFL,NBA,NHL,MLB
NFL,1.000000,0.972342,0.045363,0.821318
NBA,0.972342,1.000000,0.144367,0.956833
NHL,0.045363,0.144367,1.000000,0.003220
MLB,0.821318,0.956833,0.003220,1.000000


In [781]:
p_values.loc["NBA", "NHL"] - 0.02 

0.12436675651787647

In [782]:
p_values.loc["MLB", "NFL"] - 0.80 <= 1e-2

False

In [783]:
print('MLB - ',stats.pearsonr(mlb_data['Population 2016'], mlb_data['win_loss_mlb'])[0])
print('NHL - ',stats.pearsonr(nhl_data['Population 2016'], nhl_data['win_loss_nhl'])[0])
print('NFL - ',stats.pearsonr(nfl_data['Population 2016'], nfl_data['win_loss_nfl'])[0])
print('NBA - ',stats.pearsonr(nba_data['Population 2016'], nba_data['win_loss_nba'])[0])
# mlb_data

MLB -  0.15027698302669307
NHL -  0.012486162921209923
NFL -  -0.008854721884910915
NBA -  -0.17657160252844614


In [785]:
mlb_data

,Population 2016,win_loss_mlb
Metropolitan area,,
Atlanta,5789700,0.555556
Baltimore,2798886,0.290123
Boston,4794447,0.666667
Chicago,9512999,0.482769
Cincinnati,2165139,0.413580
Cleveland,2055612,0.561728
Dallas–Fort Worth,7233323,0.413580
Denver,2853077,0.558282
Detroit,4297617,0.395062


In [786]:
nfl_data

,Population 2016,win_loss_nfl
Metropolitan area,,
Atlanta,5789700,0.437500
Baltimore,2798886,0.625000
Boston,4794447,0.687500
Buffalo,1132804,0.375000
Charlotte,2474314,0.437500
Chicago,9512999,0.750000
Cincinnati,2165139,0.375000
Cleveland,2055612,0.466667
Dallas–Fort Worth,7233323,0.625000


In [787]:
print(nhl_nba.shape)
print(nhl_mlb.shape)
print(nhl_nfl.shape)
print(nfl_nba.shape)
print(nfl_mlb.shape)
print(nba_mlb.shape)
print(nba_data.shape)
print(mlb_data.shape)
print(nfl_data.shape)
print(nhl_data.shape)



(14, 4)
(17, 4)
(17, 4)
(19, 4)
(22, 4)
(18, 4)
(28, 2)
(26, 2)
(29, 2)
(28, 2)
